# Runoff anomaly
***

***Author:** Chus Casado Rodríguez*<br>
***Date:** 27-01-2025*<br>

**Introduction**<br>

This notebook computes the runoff anomaly in the year 2024 based on the results of the GloFAS4 historical run. The reference period used as climatology spans from 1991 to 2020.

**Output**<br>
A NetCDF of runoff anomaly.

In [1]:
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from datetime import datetime

## Configuration

In [2]:
VAR = 'runoff'
PATH = Path('../data/')
CLIMA_FILE = PATH / VAR / f'{VAR}_avg_1991-2020.nc'
YEAR_FILE = PATH / VAR / f'{VAR}_avg_2024.nc'

In [3]:
map_variables = {
    'runoff': 'rowe',
    'discharge': 'dis24'
}

var_shortname = map_variables[VAR]

## Data

In [4]:
# load climatology: average period 1991-2020
climatology = xr.open_dataset(CLIMA_FILE)[var_shortname]
climatology.close()

In [5]:
# load year of analysis: 2024
data = xr.open_dataset(YEAR_FILE)[var_shortname]
data.close()

## Anomaly

In [6]:
# compute annual anomaly
anomaly = (data - climatology) * 365 # mm/yr

In [7]:
# correct name and coordinates
anomaly.name = VAR
anomaly = anomaly.drop_vars('surface', errors='ignore')

In [8]:
# add attributes
anomaly.attrs['long_name'] = 'runoff anomaly'
anomaly.attrs['units'] = 'mm/yr'
anomaly.attrs['climatology'] = '1991-2020'
anomaly.attrs['source'] = 'GloFASv4'
anomaly.attrs['crs'] = 'epsg:4326'
anomaly.attrs['author'] = 'Jesús Casado Rodríguez <jesus.casado-rodriguez@ec.europa.eu>'
anomaly.attrs['institution'] = 'Joint Research Centre - European Commission'
anomaly.attrs['history'] = 'Created {0}'.format(datetime.now().strftime("%B %d %Y %H:%M:%S"))

### Plot

In [9]:
# boundaries = [-1e3, -500, -250, -100, -50, -25, 25, 50, 100, 250, 500, 1e3]
# cmap = plt.cm.BrBG
# norm = mcolors.BoundaryNorm(boundaries, cmap.N)#, extend='both')

# # fig, ax = plt.subplots(figsize=(12, 6), subplot_kw=dict(projection=ccrs.PlateCarree()))
# anomaly.plot(
#     cmap=cmap,
#     norm=norm,
#     cbar_kwargs={
#         'orientation': 'horizontal', 
#         'pad': 0.05, 
#         'label': 'Anomaly',
#         'ticks': [-500, -250, -100, -50, 25, 50, 100, 250, 500], 
#     }
# )
# ax.axis('off')

# plt.show()

In [10]:
# export
anomaly.to_netcdf(PATH / VAR / f'{VAR}_anomaly.nc')